<a href="https://colab.research.google.com/github/alecoder1/PyTorch-Tutorial/blob/main/Tensor_ops_(Reshape_%26_View).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Differences between Reshape and View

https://stackoverflow.com/questions/79025552/pytorch-difference-between-reshape-and-view-method/79026251#79026251

In [44]:
x = torch.tensor([1, 2, 3, 4, 5], dtype = torch.float32)
x = x.reshape(-1, 1)
x

tensor([[1.],
        [2.],
        [3.],
        [4.],
        [5.]])

In [46]:
x = x.view(x.shape[0], 1)
x

tensor([[1.],
        [2.],
        [3.],
        [4.],
        [5.]])

- The main difference is how torch.Tensor.reshape() and torch.Tensor.view() handle non-contiguous tensors.
- To understand the difference, we need to understand what is a contiguous tensor, and what is a view of a tensor:
- A contiguous tensor is a tensor whose values are stored in a single, uninterrupted – thus, "contiguous" – piece of memory. A non-contiguous tensor may have gaps in its memory layout.
- Producing a view of a tensor means reinterpreting the arrangement of values in its memory. Think of a piece of memory that stores 16 values: we can interpret it, for example, as a 16-element 1-d tensor or as a 4×4-element 2-d tensor. Both interpretations can use the same underlying memory. By using views and thus reinterpreting the memory layout, we can create differently shaped tensors from the same piece of memory, in this way avoiding duplication and saving memory.
Now back to the two methods:

If applied to a contiguous tensor, both reshape() and view() will produce a new view of the given tensor's memory, in this way avoiding duplication.
If applied to a non-contiguous tensor, the creation of a view will not be possible. The two methods handle this situation differently:
The reshape() method will duplicate the necessary piece of memory and will return a tensor whose memory will not be shared with the given tensor.
The view() method will produce a RuntimeError.
We can demonstrate this with the following piece of code:


In [50]:
from torch import arange

# Create contiguous 4×4 tensor
contiguous = arange(16).view(4, 4)
contiguous

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15]])

In [51]:
# Create non-contiguous 4×4 tensor
noncontiguous = arange(20).view(4, 5)[:, :4]
noncontiguous

tensor([[ 0,  1,  2,  3],
        [ 5,  6,  7,  8],
        [10, 11, 12, 13],
        [15, 16, 17, 18]])

In [52]:
contiguous_r = contiguous.reshape(16)
# OK: produces a 1-d view
contiguous_r

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [59]:
# Same memory used
assert contiguous_r.data_ptr() == contiguous.data_ptr() # Same memory used
contiguous_r

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [60]:
contiguous_v = contiguous.view(16)
# OK: produces a 1-d view
contiguous_v

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [66]:
# Same memory used
assert contiguous_v.data_ptr() == contiguous.data_ptr()
contiguous

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15]])

In [67]:
noncontiguous_r = noncontiguous.reshape(16)
# OK: produces a new 1-d array
noncontiguous_r

tensor([ 0,  1,  2,  3,  5,  6,  7,  8, 10, 11, 12, 13, 15, 16, 17, 18])

In [68]:
# New memory used
assert noncontiguous_r.data_ptr() != noncontiguous.data_ptr()
noncontiguous

tensor([[ 0,  1,  2,  3],
        [ 5,  6,  7,  8],
        [10, 11, 12, 13],
        [15, 16, 17, 18]])

In [69]:
noncontiguous_v = noncontiguous.view(16)
noncontiguous_v

RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

- The last line will produce RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.
- A tensor's stride is: in essence, it is the information that tells us how to map the tensor's indexes to its underlying memory.